Notebook to tryout categorical embeddings 
https://github.com/Shivanandroy/CategoricalEmbedder/blob/master/example_notebook/Example%20Notebook.ipynb

Conclusion: didn't work better than onehot encoding (0.118 vs 0.115). Try later again maybe.

In [1]:
import pandas as pd
import numpy as np
import pickle
import altair as alt
%load_ext autoreload
%autoreload 2

from utils.sklearn_custom_steps import DFSimpleImputer, DFOneHotEncoder,DFMinMaxScaler,DFColumnTransformer,DFOutlierExtractor,DFOutlierExtractor,DFStandardScaler,DFRobustScaler,DFSmartImputer, DFUnSkewer, DFPowerTransformer
from utils.sklearn_custom_steps import get_pipeline
from utils.model_hyperparameters import models

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet,SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import cross_validate
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from utils.model_hyperparameters import AutoCatBoostRegressor

def load(filename):
    f = open(filename,"rb")
    return pickle.load(f)
    
def save(model, filename='bestmodel.pickle'):
    with open('output/'+filename, 'wb') as handle:
        pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

def save_feature_selection(cols, filename='feat_selection.pickle'):
    with open('output/'+filename, 'wb') as handle:
        pickle.dump(cols, handle, protocol=pickle.HIGHEST_PROTOCOL)

def submit(model, filename='submission.csv'):
    pred = model.predict(final_test)
    final_test['SalePrice'] = np.exp(pred)
    final_test[['Id','SalePrice']].to_csv('output/'+filename, index=False)

In [2]:
import categorical_embedder as ce
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
f = open("output/engineered_datasets.pickle","rb")
train_x, train_y, final_test, _,_,_ = pickle.load(f)

#imputing manually
for col in train_x.columns:
    if train_x[col].dtype.name == 'category':
        train_x[col] = train_x[col].cat.add_categories('Unknown')
        train_x[col].fillna('Unknown',inplace=True)
    else:
        train_x[col].fillna(0,inplace=True)
# select categorical columns for embedding
from sklearn.compose import make_column_selector
num_x = make_column_selector(dtype_include=np.number)(train_x)
cat_x = make_column_selector(dtype_exclude=np.number)(train_x)

def cross_val_models(to_test,train_x=train_x,**kwargs):
    for name in to_test:
        print(f"{name.ljust(20)}", end = ': ')
        pipe = get_pipeline(models[name].model, **models[name].preprocess, **kwargs)
        test_pipeline(pipe, train_x = train_x)
         
def test_model(model,train_x = train_x,param=None):
    if not param: param = {}
    pipe = get_pipeline(model,**param)
    return test_pipeline(pipe, train_x=train_x)

def test_pipeline(pipe,train_x = train_x):
    # print(train_x.shape)
    num_fold = 5
    
    scores = cross_validate(pipe, train_x, train_y, scoring='neg_root_mean_squared_error', cv=num_fold, return_train_score=True)
    print(f"test {-1 * sum(scores['test_score'])/num_fold:.7f}, train {-1 * sum(scores['train_score'])/num_fold:.7f}")
    return pipe

In [4]:
# ce.get_embedding_info identifies the categorical variables, # of unique values and embedding size and returns a dictionary
embedding_info = ce.get_embedding_info(train_x,categorical_variables=cat_x)

In [5]:
# ce.get_label_encoded_data integer encodes the categorical variables and prepares it to feed it to neural network
X_encoded,encoders = ce.get_label_encoded_data(train_x,categorical_variables=cat_x)

In [6]:
# ce.get_embeddings trains NN, extracts embeddings and return a dictionary containing the embeddings
# did some fiddling around with the package: needed to change metric to MAE, and add 2 layers
# changed initializer to glorot instead of normal
embeddings = ce.get_embeddings(X_encoded,train_y, categorical_embedding_info=embedding_info, 
                            is_classification=False, epochs=200,batch_size=256)

Epoch 1/200
5/5 [==============================] - 3s 665ms/step - loss: 2534782208.0000 - mean_absolute_error: 2534782208.0000 - val_loss: 654931008.0000 - val_mean_absolute_error: 654931008.0000
Epoch 2/200
5/5 [==============================] - 1s 174ms/step - loss: 320620768.0000 - mean_absolute_error: 320620768.0000 - val_loss: 38221344.0000 - val_mean_absolute_error: 38221344.0000
Epoch 3/200
5/5 [==============================] - 1s 161ms/step - loss: 111956352.0000 - mean_absolute_error: 111956352.0000 - val_loss: 97389752.0000 - val_mean_absolute_error: 97389752.0000
Epoch 4/200
5/5 [==============================] - 0s 79ms/step - loss: 81844696.0000 - mean_absolute_error: 81844696.0000 - val_loss: 214066784.0000 - val_mean_absolute_error: 214066784.0000
Epoch 5/200
5/5 [==============================] - 0s 91ms/step - loss: 151299728.0000 - mean_absolute_error: 151299728.0000 - val_loss: 207280416.0000 - val_mean_absolute_error: 207280416.0000
Epoch 6/200
5/5 [==============

In [7]:
# reference
test_model(Lasso(alpha=0.0005304432735934807))

test 0.1158836, train 0.0971501


Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=Memory(location=C:\Users\Jesse\AppData\Local\Temp\tmpc_j4nbrz\joblib),
                          steps=[('col_trans',
                                  DFColumnTransformer(n_jobs=None,
                                                      remainder='drop',
                                                      sparse_threshold=0.3,
                                                      transformer_weights=None,
                                                      transformers=[('numeric',
                                                                     Pipeline(memory=None,
                                                                              steps=[('impute_num',
                                                                                      DFSimpleImputer(add_indicator=False, copy...
                                                                              verbose=False),
     

In [8]:
# now test with embeddings for categorical values instead of onehot
data = ce.fit_transform(train_x, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
test_model(Lasso(alpha=0.0005304432735934807),train_x=data)


test 0.1182161, train 0.0966422


Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=Memory(location=C:\Users\Jesse\AppData\Local\Temp\tmpc_j4nbrz\joblib),
                          steps=[('col_trans',
                                  DFColumnTransformer(n_jobs=None,
                                                      remainder='drop',
                                                      sparse_threshold=0.3,
                                                      transformer_weights=None,
                                                      transformers=[('numeric',
                                                                     Pipeline(memory=None,
                                                                              steps=[('impute_num',
                                                                                      DFSimpleImputer(add_indicator=False, copy...
                                                                              verbose=False),
     